<br/>

<img src="https://uploads-ssl.webflow.com/614b1fe22fa8b90ef41aeffe/6265cb48f9496b1cefc9ab75_logotipo-mbit-39.png" width="200px" align="right" CLASS="TextWrap" style="background-color:#2a3f3f;">

<h1><font color="#2a3f3f" size=5>Cabifly APP</font></h1>

<br/>
<div style="text-align: right">
<font color="#2a3f3f" size=3>Adrian Sanchez - adrian.sanchez@mbitschool.com</font><br>
<font color="#2a3f3f" size=3>Máster en Data Engineering </font><br>
</div>


¡La App principal de Cabifly!

Esta libreta simula la aplicación de cabifly utilizando la librería `ipyleaftlet` una versión interactiva de Folium.

En esta libreta:

- Utilizamos la API de drones para obtener su ubicación y pintarlos de manera interactiva en el mapa
- Utilizamos la API de trips para solicitar un nuevo viaje y determinar la ubicación del drone más idóneo

__TAREAS:__

- No tienes que hacer nada en esta libreta, solo comprobar que funciona con el resto de componentes que estás construyendo!
- Acuérdate de actualizar las URLs cuando quieras utilizar tus microservicios en AWS

## Configuración de la interfaz

Utilizamos el paquete ipyleaflet para que el mapa sea interactivo

In [1]:
pip install ipyleaflet

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -------------------------------------- - 133.1/139.4 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.1 MB 15.2 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 13.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/83.9 kB ? eta -:--:--
   ---------------------------------------- 83.9/83.9 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   ---------------------------------------- 214.4/214.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------  2.3/2.3 MB 75.0 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 49.5 MB/s eta 0:00:00
Note: you may need to restart t


[notice] A new release of pip is available: 23.3.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

from ipyleaflet import Map, Marker, Icon

import requests
import time

In [2]:
geo_madrid = (40.4168, -3.7038)

In [3]:
m = Map(
    center=geo_madrid, 
    zoom=15
)

Esta función convierte los datos de los drones en marcadores del mapa

In [4]:
def get_marker(drone):
    return Marker(
        icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/1830/1830867.png', icon_size=[40, 40], icon_anchor=[40,40]),
        location=(drone["location"]["coordinates"][1], drone["location"]["coordinates"][0]),
        title=drone["drone_id"],
        draggable=False
    )

### Carga de datos inicial

Cargamos los drones de la API y los añadimos al mapa

In [8]:
# Get drones from mongodb
drones = requests.get("https://4rf1docx24.execute-api.us-east-1.amazonaws.com/dev/drones", params={"lon":-3.7084288946607136, "lat":40.41702820529783, "distance": 1000}).json()

In [9]:
markers = {d["drone_id"]: get_marker(d) for d in drones}

In [10]:
for mkr in markers.values():
    m.add_layer(mkr)

### Marcador para posición del usuario

Permitirá obtener su posición para llamar a la API de trips

In [11]:
user_location = Marker(location=geo_madrid)
m.add_layer(user_location)

In [12]:
user_location.location

[40.4168, -3.7038]

### Mostramos y actualizamos el mapa

In [29]:
m

Map(center=[40.4168, -3.7038], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

### Llamadas a la API para crear y consultar los viajes

In [16]:
trip = requests.post(
    "https://4rf1docx24.execute-api.us-east-1.amazonaws.com/dev/users/jesus_3/trips", 
    json={
        "lon": user_location.location[1], 
        "lat": user_location.location[0]
    }
).json()

In [17]:
trip

{'status': 'Trip created',
 'trip': {'created_at': '2024-07-04T12:20:45.976418',
  'location': [-3.7038, 40.4168],
  'status': 'waiting',
  'trip_id': 'b53ab9e0-6697-44b5-8566-1c6cf5a9399e',
  'user_id': 'jesus_3'},
 'trip_id': 'b53ab9e0-6697-44b5-8566-1c6cf5a9399e'}

In [18]:
# Get drones from mongodb
trips = requests.get("https://4rf1docx24.execute-api.us-east-1.amazonaws.com/dev/users/jesus_3/trips").json()

In [19]:
trips

[{'created_at': '2024-07-04T12:15:00.846388',
  'drone_id': 'af021647-8aad-4077-bff5-1ee9b9a62d81',
  'location': [-3.7084288946607136, 40.41702820529783],
  'status': 'accepted',
  'trip_id': 'c4c320e2-1c47-4c65-a602-0d19a6e2d995',
  'user_id': 'jesus_3'},
 {'created_at': '2024-07-04T12:20:45.976418',
  'drone_id': '7de4e10f-a9ab-4fc5-a56a-6c0df56de1b5',
  'location': [-3.7038, 40.4168],
  'status': 'accepted',
  'trip_id': 'b53ab9e0-6697-44b5-8566-1c6cf5a9399e',
  'user_id': 'jesus_3'}]

### Bucle de actualización del mapa

Lo ejecutamos en un thread para no bloquear la libreta

In [20]:
import threading

In [31]:
# Para pararlo cambiar esta variable
running = False

In [25]:
def update_map():
    while running:
        # Get drones from mongodb
        # drones = requests.get()
        drones = requests.get("https://4rf1docx24.execute-api.us-east-1.amazonaws.com/dev/drones", params={"lon":-3.7084288946607136, "lat":40.41702820529783, "distance": 1000}).json()

        for d in drones:
            markers[d["drone_id"]].location = (d["location"]["coordinates"][1], d["location"]["coordinates"][0])

        time.sleep(5)

In [26]:
thread = threading.Thread(target=update_map)

In [27]:
thread.start()

In [30]:
thread.is_alive()

True